In [ ]:
import os
import sys
from datetime import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from numba import njit
import numpy as np
from lib.utils import create_windows, file_to_data_frame, ExtendedPortfolio
import pandas as pd
import vectorbt as vbt

In [ ]:
# leemos el csv
(s_name, ohlcv) = file_to_data_frame(
    "../../Binance_ADAUSDT_minute.csv")
# agarramos solo las columnas que necesitamos
cols = ohlcv.columns
print(cols)
#ohlcv.get(["Open", "High", "Low", "Close", "Volume"]).vbt.ohlcv.plot().show_png()
ohlc = ohlcv.get(["Open", "High", "Low", "Close"])
print(ohlc.head())

In [ ]:
# creamos las ventanas
figure, windows = create_windows(ohlc=ohlc, n=8, window_len=0.6, training_set_len=0.4)
(in_df, in_df_index), (out_df, _) = windows

In [ ]:
figure.show()

In [ ]:
in_df.head()

In [ ]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m'
)


In [ ]:
# creamos el indicador
@njit
def apply_alpha_nb(open: np.ndarray, high: np.ndarray, low: np.ndarray, close: np.ndarray, buy_threshold: float,
                  sell_threshold: float):
    aux = (close - open) / (high - low + 0.001)
    aux = np.where(aux >= buy_threshold, 1, aux)
    aux = np.where(aux <= -sell_threshold, -1, aux)
    return aux

AlphaInd = vbt.IndicatorFactory(
    input_names=['open', 'high', 'low', 'close'],
    param_names=['buy_threshold', 'sell_threshold'],
    output_names=['signal']
).from_apply_func(apply_alpha_nb)
# dir(AlphaInd)

In [ ]:
# Corremos con todas las combinaciones
def simulate_all_params(ohlc_windows, buy_range, sell_range):
    # creamos las señales
    open = ohlc_windows.xs("Open", level=1, axis=1)
    high = ohlc_windows.xs("High", level=1, axis=1)
    low = ohlc_windows.xs("Low", level=1, axis=1)
    close = ohlc_windows.xs("Close", level=1, axis=1)
    momentum = AlphaInd.run(open=open, high=high, low=low, close=close,
                            buy_threshold=buy_range, sell_threshold=sell_range,
                            param_product=True,
                            short_name="alpha")
    ones = np.full(momentum.signal.shape[-1], 1)
    entry_signal = momentum.signal_equal(ones)
    exit_signal = momentum.signal_equal(-ones)
    return ExtendedPortfolio.from_signals(close, entry_signal, exit_signal, **portfolio_kwargs)


# Optimizamos para el in y el out
buy_range = np.linspace(0.0, 6, 10, endpoint=False)
sell_range = np.linspace(0.7, 1, 10, endpoint=False)
in_port = simulate_all_params(in_df, buy_range, sell_range)
in_sharpe = in_port.sharpe_ratio()
in_elr = in_port.expected_log_returns()
#out_elr = simulate_all_params(out_df, params_range).expected_log_returns()

In [ ]:
def get_best_index(performance, higher_better=True):
    if higher_better:
        return performance[performance.groupby('split_idx').idxmax()].index
    return performance[performance.groupby('split_idx').idxmin()].index

def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [ ]:
#Buscamos el índice de los mejores resultados del in
in_best_index = get_best_index(in_elr)
print(in_best_index)

In [ ]:
in_best_buy_thresholds = get_best_params(in_best_index, 'alpha_buy_threshold')
in_best_sell_thresholds = get_best_params(in_best_index, 'alpha_sell_threshold')
in_best_threshold_pairs = np.array(list(zip(in_best_buy_thresholds, -in_best_sell_thresholds)))

print(in_best_threshold_pairs)

In [ ]:
pd.DataFrame(in_best_threshold_pairs, columns=['buy_threshold', 'sell_threshold']).vbt.plot().show()

In [ ]:
# Corremos el out con los mejores parámetros de in
#close, high, low, open = list(map(lambda tu: tu[1], in_df.groupby(level=1, axis=1)))
open = in_df.xs("Open", level=1, axis=1)
high = in_df.xs("High", level=1, axis=1)
low = in_df.xs("Low", level=1, axis=1)
close = in_df.xs("Close", level=1, axis=1)
momentum = AlphaInd.run(open=open, high=high, low=low, close=close,
                        buy_threshold=in_best_buy_thresholds, sell_threshold=in_best_sell_thresholds,
                        short_name="alpha", per_column=True)
ones = np.full(momentum.signal.shape, 1)
entry_signal = momentum.signal_equal(ones, crossover=True)
exit_signal = momentum.signal_equal(-ones, crossover=True)
# imprimo para confirmar que haya algún true
entry_signal.loc[entry_signal[in_best_buy_thresholds[0], in_best_sell_thresholds[0], 0]==True].head()

In [ ]:
trade_price = close
out_test_port = ExtendedPortfolio.from_signals(trade_price, entry_signal, exit_signal, **portfolio_kwargs)
out_test_elr = out_test_port.expected_log_returns()
out_test_sharpe = out_test_port.sharpe_ratio()
print(out_test_elr)

In [ ]:
# simulamos Buy&Hold de cada in y out window y tomamos el expected log returns (elr)
close_columns = list(filter(lambda col: "Close" in col[1], in_df.columns))
in_hold_port = ExtendedPortfolio.from_holding(in_df[close_columns], **portfolio_kwargs)
out_hold_port = ExtendedPortfolio.from_holding(out_df[close_columns], **portfolio_kwargs)
print(in_hold_port.trades.values[:4])
print(out_hold_port.trades.values[:4])
in_hold_elr = in_hold_port.expected_log_returns()
in_hold_sharpe = in_hold_port.sharpe_ratio()
out_hold_elr = out_hold_port.expected_log_returns()
out_hold_sharpe = out_hold_port.sharpe_ratio()
print(in_hold_elr, out_hold_elr)

In [ ]:
# ploteamos los elrs
cv_results_df = pd.DataFrame({
    'in_sample_hold': in_hold_elr.values,
    #'in_sample_median': in_elr.groupby('split_idx').median().values,
    'in_sample_best': in_elr[in_best_index].values,
    'out_sample_hold': out_hold_elr.values,
    #'out_sample_median': out_elr.groupby('split_idx').median().values,
    'out_sample_test': out_test_elr.values
})

cv_results_df.vbt.plot(
    trace_kwargs=[
        dict(line_color=vbt.settings.color_schema['blue']),
        ##dict(line_color=vbt.settings.color_schema['blue'], line_dash='dash'),
        dict(line_color=vbt.settings.color_schema['blue'], line_dash='dot'),
        dict(line_color=vbt.settings.color_schema['orange']),
        #dict(line_color=vbt.settings.color_schema['orange'], line_dash='dash'),
        dict(line_color=vbt.settings.color_schema['orange'], line_dash='dot')
    ]
).show()

In [ ]:
#ploteamos los sharpes
cv_results_df = pd.DataFrame({
    'in_sample_hold': in_hold_sharpe.values,
    'in_sample_best': in_sharpe[in_best_index].values,
    'out_sample_hold': out_hold_sharpe.values,
    'out_sample_test': out_test_sharpe.values
})

cv_results_df.vbt.plot(
    trace_kwargs=[
        dict(line_color=vbt.settings.color_schema['blue']),
        dict(line_color=vbt.settings.color_schema['blue'], line_dash='dot'),
        dict(line_color=vbt.settings.color_schema['orange']),
        dict(line_color=vbt.settings.color_schema['orange'], line_dash='dot')
    ]
).show()

In [ ]:
for col in in_best_index:
    out_test_port.trades.plot(column=col).show()
out_test_port.trades.plot_pnl(column=col).show()

